# Mass Transports

Transport diagnostics for flow through major straits.

## Theory

Formally, mass transports are given by
$$T_x = \rho u  $$
$$T_y = \rho v  $$

Mass transports are diagnostics that are calculated online by the model:

|variable|long name|units|dimensions|
|-|
|tx_trans|T-cell i-mass transport|Sv|(time,st_ocean,xu_ocean,yt_ocean)|
|ty_trans|T-cell j-mass transport|Sv|(time,st_ocean,xt_ocean,yu_ocean)|


These variables are saved in `ocean.nc` files.

## Calculation

In [1]:
%matplotlib inline
import cosima_cookbook as cc
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.basemap import Basemap, shiftgrid
import numpy as np
import netCDF4 as nc
import xarray as xr
import cmocean as cm
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cft
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import pandas as pd
import copy
import math
from collections import OrderedDict
import IPython.display
from dask.distributed import Client
import cartopy.crs as ccrs
def arange(start,end,stride=1):
    return np.arange(start,end+0.00000001,stride)

import sys, os
sys.path.append(os.path.join(os.getcwd(), '..'))  # so we can import ../exptdata
import exptdata
print('Available exptdata keys: ', [k for k in exptdata.exptdict.keys()])


netcdf_index loaded.
Available exptdata keys:  ['1deg', '025deg', '01deg']


In [2]:
#cc.start_cluster()
client = Client()
client

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-18.10/lib/python3.6/site-packages/distributed/bokeh/core.py:57: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


Client Scheduler: tcp://127.0.0.1:46640 Dashboard: http://127.0.0.1:37966/status,Cluster Workers: 8 Cores: 8 Memory: 33.67 GB


In [3]:
%%javascript
IPython.notebook.kernel.execute('nb_name = ' + '"' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [4]:
# use common start and end dates for all runs
tstart = exptdata.clim_tstart
tend = exptdata.clim_tend

lonW = 90.0
lonE = 180.0
latS = -70.0
latN = 20.0

firstyear = pd.to_datetime(tstart).year  # assumes tstart is 1 January!
lastyear = pd.to_datetime(tend).year-1  # assumes tend is 1 January!
yearrange = str(firstyear)+'-'+str(lastyear)
print(yearrange)
print('tstart = ', tstart)
print('tend = ', tend)

output_dir = '/home/561/erd561/figures/Australia/'
fig_name = nb_name[:-6]
fig_dir = output_dir + fig_name

output_ls = os.listdir(output_dir)
if fig_name not in output_ls:
    os.mkdir(fig_dir) 

1993-2017
tstart =  1993-01-01 00:00:00
tend =  2018-01-01 00:00:00


In [5]:
input_path_obs = '/g/data/e14/erd561/NOAA_OI_SST_V2/'

input_path_mod = '/g/data/e14/erd561/access-om2/'

In [7]:
straits_01deg = OrderedDict([ # use OrderedDict so panels are produced in a reproducible order
            ('Lombok Strait', [-244.3, -243.8, -8.37, -8.37]),
            ('Ombai Strait',  [-234.9, -234.9, -8.86, -8.27]),
            ('Timor Strait',  [-232.6, -232.6, -13.96, -8.37])
            ])
straits_025deg = OrderedDict([ # use OrderedDict so panels are produced in a reproducible order
            ('Lombok Strait', [-244.5, -243.5, -8.22, -8.22]),
            ('Ombai Strait',  [-235.5, -235.5, -9.21, -8.47]),
            ('Timor Strait',  [-233, -233, -13.86, -8.71])
            ])
straits_1deg = OrderedDict([ # use OrderedDict so panels are produced in a reproducible order
            ('Lombok Strait', [-245, -242, -8.33, -8.33]),
            ('Ombai Strait',  [-237, -237, -10.33, -8.33]),
            ('Timor Strait',  [-233, -233, -13.9, -9])
            ])


lonW_LC = 110
lonE_LC = 119
latS_LC = -34
latN_LC = -20

latN_157E = -20

lonW_EAC = 147
lonE_EAC = 157
latS_EAC = -47
latN_EAC = -33


LC = OrderedDict([ # use OrderedDict so panels are produced in a reproducible order
            ('LC_north Strait', [lonW_LC-360, lonE_LC-360, latN_LC, latN_LC]),
            ('LC_west Strait', [lonW_LC-360, lonW_LC-360, latS_LC, latN_LC]),
            ('LC_south Strait', [lonW_LC-360, lonE_LC-360, latS_LC, latS_LC])
            ])

EAC = OrderedDict([ # use OrderedDict so panels are produced in a reproducible order
            ('EAC_north Strait', [lonW_EAC-360, lonE_EAC-360, latN_EAC, latN_EAC]),
            ('EAC_east Strait', [lonE_EAC-360, lonE_EAC-360, latS_EAC, latN_EAC]),
            ('EAC_south Strait', [lonW_EAC-360, lonE_EAC-360, latS_EAC, latS_EAC]),
            ('EAC_west Strait', [lonW_EAC-360, lonW_EAC-360, latS_EAC, -42.2])
            ])

line_157E = OrderedDict([ # use OrderedDict so panels are produced in a reproducible order
            ('line_157E Strait', [lonE_EAC-360, lonE_EAC-360, latN_EAC, latN_157E])
            ])


In [8]:
input_path_obs = '/g/data/e14/erd561/NOAA_OI_SST_V2/'

output_path_obs = '/g/data/e14/erd561/NOAA_OI_SST_V2/'
output_path_mod = '/g/data/e14/erd561/access-om2/'

In [11]:
ekey='1deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_hov_LC_north_1 = cc.get_nc_variable(expt, 'ocean_month.nc', 'ty_trans_int_z',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xt_ocean=slice(
                                    LC['LC_north Strait'][0],
                                    LC['LC_north Strait'][1])).\
                        sel(yu_ocean=LC['LC_north Strait'][2], method='nearest')

trans_hov_LC_north_1.load()

trans_hov_LC_north_1 = trans_hov_LC_north_1.groupby('time.year').mean('time', skipna=True)
print(trans_hov_LC_north_1)


ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_hov_LC_north_025 = cc.get_nc_variable(expt, 'ocean_month.nc', 'ty_trans_int_z',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xt_ocean=slice(
                                    LC['LC_north Strait'][0],
                                    LC['LC_north Strait'][1])).\
                        sel(yu_ocean=LC['LC_north Strait'][2], method='nearest')

trans_hov_LC_north_025.load()

trans_hov_LC_north_025 = trans_hov_LC_north_025.groupby('time.year').mean('time', skipna=True)
print(trans_hov_LC_north_025)


ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_hov_LC_north_01_month = cc.get_nc_variable(expt, 'ocean_month.nc', 'ty_trans_int_z',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xt_ocean=slice(
                                    LC['LC_north Strait'][0],
                                    LC['LC_north Strait'][1])).\
                        sel(yu_ocean=LC['LC_north Strait'][2], method='nearest')

trans_hov_LC_north_01_month.load()
print(trans_hov_LC_north_01_month)

trans_hov_LC_north_01 = trans_hov_LC_north_01_month.groupby('time.year').mean('time', skipna=True)
print(trans_hov_LC_north_01)


time_units = 'days since 0001-01-01'
print(time_units)

trans_hov_LC_north_JRA_month = cc.get_nc_variable('01deg_jra55v13_ryf8485_spinup6', 'ocean_month.nc', 'ty_trans_int_z',
                        time_units=time_units).\
                        sel(xt_ocean=slice(
                                    LC['LC_north Strait'][0],
                                    LC['LC_north Strait'][1])).\
                        sel(yu_ocean=LC['LC_north Strait'][2], method='nearest')

trans_hov_LC_north_JRA_month.load()



trans_hov_LC_north_JRA1 = trans_hov_LC_north_JRA_month.isel(time=slice(0,418)).\
                        groupby('time.year').mean('time', skipna=True)
trans_hov_LC_north_JRA1 = trans_hov_LC_north_JRA1.assign_coords(year = trans_hov_LC_north_JRA1.year.values + 1944)
print(trans_hov_LC_north_JRA1.year.values)



trans_hov_LC_north_JRA2 = trans_hov_LC_north_JRA_month.isel(time=slice(446,-8))
trans_hov_LC_north_JRA2 = trans_hov_LC_north_JRA2.assign_coords(time = pd.to_datetime(trans_hov_LC_north_JRA2.time.values))
trans_hov_LC_north_JRA2 = trans_hov_LC_north_JRA2.groupby('time.year').mean('time', skipna=True)
print(trans_hov_LC_north_JRA2.year.values)
trans_hov_LC_north_JRA2 = trans_hov_LC_north_JRA2.assign_coords(year = trans_hov_LC_north_JRA2.year.values + 44)
print(trans_hov_LC_north_JRA2.year.values)



trans_hov_LC_north_JRA = xr.concat([trans_hov_LC_north_JRA1, trans_hov_LC_north_JRA2], dim='year')
print(trans_hov_LC_north_JRA)

trans_hov_LC_north_1_xr = xr.DataArray(trans_hov_LC_north_1, name='trans_hov_LC_north_1',
                        coords=[trans_hov_LC_north_1.year], 
                        dims=['year'])
print(trans_hov_LC_north_1_xr)
trans_hov_LC_north_1_xr.to_netcdf(output_path_mod + 'trans_hov_LC_north_1.nc')


trans_hov_LC_north_025_xr = xr.DataArray(trans_hov_LC_north_025, name='trans_hov_LC_north_025',
                        coords=[trans_hov_LC_north_025.year], 
                        dims=['year'])
print(trans_hov_LC_north_025_xr)
trans_hov_LC_north_025_xr.to_netcdf(output_path_mod + 'trans_hov_LC_north_025.nc')


trans_hov_LC_north_01_xr = xr.DataArray(trans_hov_LC_north_01, name='trans_hov_LC_north_01',
                        coords=[trans_hov_LC_north_01.year], 
                        dims=['year'])
print(trans_hov_LC_north_01_xr)
trans_hov_LC_north_01_xr.to_netcdf(output_path_mod + 'trans_hov_LC_north_01.nc')


trans_hov_LC_north_JRA_xr = xr.DataArray(trans_hov_LC_north_JRA, name='trans_hov_LC_north_JRA',
                        coords=[trans_hov_LC_north_JRA.year], 
                        dims=['year'])
print(trans_hov_LC_north_JRA_xr)
trans_hov_LC_north_JRA_xr.to_netcdf(output_path_mod + 'trans_hov_LC_north_JRA.nc')

1993-01-01 00:00:00
2018-01-01 00:00:00
1deg_jra55v13_iaf_spinup1_B1
-12
days since 1718-01-01
-87658
Using database sqlite:////g/data3/hh5/tmp/cosima/cosima-cookbook/cosima-cookbook.db
<xarray.DataArray 'ty_trans_int_z' (year: 300, xt_ocean: 9)>
array([[-2.073008e+08,  2.281929e+07,  1.127992e+08, ..., -4.124141e+07,
        -2.177368e+07, -4.842059e+07],
       [-1.320710e+08, -2.589535e+08, -2.305204e+08, ..., -3.857125e+07,
        -2.112017e+07, -4.791705e+07],
       [ 1.309024e+08, -1.714617e+07, -1.216235e+08, ..., -6.659904e+07,
        -3.246600e+07, -7.285118e+07],
       ...,
       [-3.980519e+07, -7.780950e+07, -6.518245e+07, ..., -5.170870e+07,
        -3.902108e+07, -5.642260e+07],
       [ 5.467710e+07, -1.496786e+08, -2.267123e+08, ..., -5.773504e+07,
        -2.730001e+07, -5.928061e+07],
       [-5.606058e+07,  1.445469e+07, -1.230293e+05, ..., -5.109451e+07,
        -3.379434e+07, -7.302962e+07]], dtype=float32)
Coordinates:
    yu_ocean  float64 -19.76
  * xt_ocea

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-18.10/lib/python3.6/site-packages/xarray/core/nanops.py:161: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


[1945 1946 1947 1948 1949 1950 1951 1952 1953 1954 1955 1956 1957 1958
 1959 1960 1961 1962 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972
 1973 1974 1975 1976 1977 1978 1979]
[1935 1936 1937 1938 1939 1940 1941 1942 1943 1944 1945]
[1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989]
<xarray.DataArray 'ty_trans_int_z' (year: 46, xt_ocean: 90)>
array([[-1.654274e+08, -1.618978e+08, -9.184009e+07, ..., -1.706569e+05,
                  nan,           nan],
       [-2.523364e+08, -2.993634e+08, -2.395627e+08, ..., -1.664672e+05,
                  nan,           nan],
       [ 7.857087e+07,  1.960765e+07, -1.353349e+07, ..., -1.789100e+05,
                  nan,           nan],
       ...,
       [-8.164058e+06, -7.602743e+07, -1.104311e+08, ..., -1.802210e+05,
                  nan,           nan],
       [-2.278512e+08, -3.046303e+08, -2.988249e+08, ..., -1.869099e+05,
                  nan,           nan],
       [ 4.282408e+07, -6.114638e+07, -1.246509e+08, ..., -1.915939e+05,
 

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-18.10/lib/python3.6/site-packages/xarray/core/nanops.py:161: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


ValueError: coords is not dict-like, but it has 1 items, which does not match the 2 dimensions of the data

In [14]:
trans_hov_LC_north_1_xr = xr.DataArray(trans_hov_LC_north_1, name='trans_hov_LC_north_1',
                        coords=[trans_hov_LC_north_1.year, trans_hov_LC_north_1.xt_ocean], 
                        dims=['year', 'xt_ocean'])
print(trans_hov_LC_north_1_xr)
trans_hov_LC_north_1_xr.to_netcdf(output_path_mod + 'trans_hov_LC_north_1.nc')


trans_hov_LC_north_025_xr = xr.DataArray(trans_hov_LC_north_025, name='trans_hov_LC_north_025',
                        coords=[trans_hov_LC_north_025.year, trans_hov_LC_north_025.xt_ocean], 
                        dims=['year', 'xt_ocean'])
print(trans_hov_LC_north_025_xr)
trans_hov_LC_north_025_xr.to_netcdf(output_path_mod + 'trans_hov_LC_north_025.nc')


trans_hov_LC_north_01_xr = xr.DataArray(trans_hov_LC_north_01, name='trans_hov_LC_north_01',
                        coords=[trans_hov_LC_north_01.year, trans_hov_LC_north_01.xt_ocean], 
                        dims=['year', 'xt_ocean'])
print(trans_hov_LC_north_01_xr)
trans_hov_LC_north_01_xr.to_netcdf(output_path_mod + 'trans_hov_LC_north_01.nc')


trans_hov_LC_north_JRA_xr = xr.DataArray(trans_hov_LC_north_JRA, name='trans_hov_LC_north_JRA',
                        coords=[trans_hov_LC_north_JRA.year, trans_hov_LC_north_JRA.xt_ocean], 
                        dims=['year', 'xt_ocean'])
print(trans_hov_LC_north_JRA_xr)
trans_hov_LC_north_JRA_xr.to_netcdf(output_path_mod + 'trans_hov_LC_north_JRA.nc')

<xarray.DataArray 'trans_hov_LC_north_1' (year: 300, xt_ocean: 9)>
array([[-2.073008e+08,  2.281929e+07,  1.127992e+08, ..., -4.124141e+07,
        -2.177368e+07, -4.842059e+07],
       [-1.320710e+08, -2.589535e+08, -2.305204e+08, ..., -3.857125e+07,
        -2.112017e+07, -4.791705e+07],
       [ 1.309024e+08, -1.714617e+07, -1.216235e+08, ..., -6.659904e+07,
        -3.246600e+07, -7.285118e+07],
       ...,
       [-3.980519e+07, -7.780950e+07, -6.518245e+07, ..., -5.170870e+07,
        -3.902108e+07, -5.642260e+07],
       [ 5.467710e+07, -1.496786e+08, -2.267123e+08, ..., -5.773504e+07,
        -2.730001e+07, -5.928061e+07],
       [-5.606058e+07,  1.445469e+07, -1.230293e+05, ..., -5.109451e+07,
        -3.379434e+07, -7.302962e+07]], dtype=float32)
Coordinates:
  * year      (year) int64 1718 1719 1720 1721 1722 ... 2013 2014 2015 2016 2017
  * xt_ocean  (xt_ocean) float64 -249.5 -248.5 -247.5 ... -243.5 -242.5 -241.5
<xarray.DataArray 'trans_hov_LC_north_025' (year: 300, xt_oc

In [ ]:
ekey='1deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_hov_LC_west_1 = cc.get_nc_variable(expt, 'ocean_month.nc', 'tx_trans_int_z',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xu_ocean=LC['LC_west Strait'][0], method='nearest').\
                        sel(yt_ocean=slice(
                                    LC['LC_west Strait'][2],
                                    LC['LC_west Strait'][3]))
                     

trans_hov_LC_west_1.load()

trans_hov_LC_west_1 = trans_hov_LC_west_1.groupby('time.year').mean('time', skipna=True)
print(trans_hov_LC_west_1)


ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_hov_LC_west_025 = cc.get_nc_variable(expt, 'ocean_month.nc', 'tx_trans_int_z',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xu_ocean=LC['LC_west Strait'][0], method='nearest').\
                        sel(yt_ocean=slice(
                                    LC['LC_west Strait'][2],
                                    LC['LC_west Strait'][3]))
                     

trans_hov_LC_west_025.load()

trans_hov_LC_west_025 = trans_hov_LC_west_025.groupby('time.year').mean('time', skipna=True)
print(trans_hov_LC_west_025)


ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_hov_LC_west_01_month = cc.get_nc_variable(expt, 'ocean_month.nc', 'tx_trans_int_z',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xu_ocean=LC['LC_west Strait'][0], method='nearest').\
                        sel(yt_ocean=slice(
                                    LC['LC_west Strait'][2],
                                    LC['LC_west Strait'][3]))
                     

trans_hov_LC_west_01_month.load()
print(trans_hov_LC_west_01_month)

trans_hov_LC_west_01 = trans_hov_LC_west_01_month.groupby('time.year').mean('time', skipna=True)
print(trans_hov_LC_west_01)


time_units = 'days since 0001-01-01'
print(time_units)

trans_hov_LC_west_JRA_month = cc.get_nc_variable('01deg_jra55v13_ryf8485_spinup6', 'ocean_month.nc', 'tx_trans_int_z',
                        time_units=time_units).\
                        sel(xu_ocean=LC['LC_west Strait'][0], method='nearest').\
                        sel(yt_ocean=slice(
                                    LC['LC_west Strait'][2],
                                    LC['LC_west Strait'][3]))

trans_hov_LC_west_JRA_month.load()



trans_hov_LC_west_JRA1 = trans_hov_LC_west_JRA_month.isel(time=slice(0,418)).\
                        groupby('time.year').mean('time', skipna=True)-273.15
trans_hov_LC_west_JRA1 = trans_hov_LC_west_JRA1.assign_coords(year = trans_hov_LC_west_JRA1.year.values + 1944)
print(trans_hov_LC_west_JRA1.year.values)



trans_hov_LC_west_JRA2 = trans_hov_LC_west_JRA_month.isel(time=slice(446,-8))
trans_hov_LC_west_JRA2 = trans_hov_LC_west_JRA2.assign_coords(time = pd.to_datetime(trans_hov_LC_west_JRA2.time.values))
trans_hov_LC_west_JRA2 = trans_hov_LC_west_JRA2.groupby('time.year').mean('time', skipna=True)
print(trans_hov_LC_west_JRA2.year.values)
trans_hov_LC_west_JRA2 = trans_hov_LC_west_JRA2.assign_coords(year = trans_hov_LC_west_JRA2.year.values + 44)
print(trans_hov_LC_west_JRA2.year.values)



trans_hov_LC_west_JRA = xr.concat([trans_hov_LC_west_JRA1, trans_hov_LC_west_JRA2], dim='year')
print(trans_hov_LC_west_JRA)

trans_hov_LC_west_1_xr = xr.DataArray(trans_hov_LC_west_1, name='trans_hov_LC_west_1',
                        coords=[trans_hov_LC_west_1.year], 
                        dims=['year'])
print(trans_hov_LC_west_1_xr)
trans_hov_LC_west_1_xr.to_netcdf(output_path_mod + 'trans_hov_LC_west_1.nc')


trans_hov_LC_west_025_xr = xr.DataArray(trans_hov_LC_west_025, name='trans_hov_LC_west_025',
                        coords=[trans_hov_LC_west_025.year], 
                        dims=['year'])
print(trans_hov_LC_west_025_xr)
trans_hov_LC_west_025_xr.to_netcdf(output_path_mod + 'trans_hov_LC_west_025.nc')


trans_hov_LC_west_01_xr = xr.DataArray(trans_hov_LC_west_01, name='trans_hov_LC_west_01',
                        coords=[trans_hov_LC_west_01.year], 
                        dims=['year'])
print(trans_hov_LC_west_01_xr)
trans_hov_LC_west_01_xr.to_netcdf(output_path_mod + 'trans_hov_LC_west_01.nc')


trans_hov_LC_west_JRA_xr = xr.DataArray(trans_hov_LC_west_JRA, name='trans_hov_LC_west_JRA',
                        coords=[trans_hov_LC_west_JRA.year], 
                        dims=['year'])
print(trans_hov_LC_west_JRA_xr)
trans_hov_LC_west_JRA_xr.to_netcdf(output_path_mod + 'trans_hov_LC_west_JRA.nc')

In [ ]:
ekey='1deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_hov_LC_south_1 = cc.get_nc_variable(expt, 'ocean_month.nc', 'ty_trans_int_z',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xt_ocean=slice(
                                    LC['LC_south Strait'][0],
                                    LC['LC_south Strait'][1])).\
                        sel(yu_ocean=LC['LC_south Strait'][2], method='nearest')

trans_hov_LC_south_1.load()

trans_hov_LC_south_1 = trans_hov_LC_south_1.groupby('time.year').mean('time', skipna=True)
print(trans_hov_LC_south_1)


ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_hov_LC_south_025 = cc.get_nc_variable(expt, 'ocean_month.nc', 'ty_trans_int_z',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xt_ocean=slice(
                                    LC['LC_south Strait'][0],
                                    LC['LC_south Strait'][1])).\
                        sel(yu_ocean=LC['LC_south Strait'][2], method='nearest')

trans_hov_LC_south_025.load()

trans_hov_LC_south_025 = trans_hov_LC_south_025.groupby('time.year').mean('time', skipna=True)
print(trans_hov_LC_south_025)


ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_hov_LC_south_01_month = cc.get_nc_variable(expt, 'ocean_month.nc', 'ty_trans_int_z',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xt_ocean=slice(
                                    LC['LC_south Strait'][0],
                                    LC['LC_south Strait'][1])).\
                        sel(yu_ocean=LC['LC_south Strait'][2], method='nearest')

trans_hov_LC_south_01_month.load()
print(trans_hov_LC_south_01_month)

trans_hov_LC_south_01 = trans_hov_LC_south_01_month.groupby('time.year').mean('time', skipna=True)
print(trans_hov_LC_south_01)


time_units = 'days since 0001-01-01'
print(time_units)

trans_hov_LC_south_JRA_month = cc.get_nc_variable('01deg_jra55v13_ryf8485_spinup6', 'ocean_month.nc', 'ty_trans_int_z',
                        time_units=time_units).\
                        sel(xt_ocean=slice(
                                    LC['LC_south Strait'][0],
                                    LC['LC_south Strait'][1])).\
                        sel(yu_ocean=LC['LC_south Strait'][2], method='nearest')

trans_hov_LC_south_JRA_month.load()



trans_hov_LC_south_JRA1 = trans_hov_LC_south_JRA_month.isel(time=slice(0,418)).\
                        groupby('time.year').mean('time', skipna=True)
trans_hov_LC_south_JRA1 = trans_hov_LC_south_JRA1.assign_coords(year = trans_hov_LC_south_JRA1.year.values + 1944)
print(trans_hov_LC_south_JRA1.year.values)



trans_hov_LC_south_JRA2 = trans_hov_LC_south_JRA_month.isel(time=slice(446,-8))
trans_hov_LC_south_JRA2 = trans_hov_LC_south_JRA2.assign_coords(time = pd.to_datetime(trans_hov_LC_south_JRA2.time.values))
trans_hov_LC_south_JRA2 = trans_hov_LC_south_JRA2.groupby('time.year').mean('time', skipna=True)
print(trans_hov_LC_south_JRA2.year.values)
trans_hov_LC_south_JRA2 = trans_hov_LC_south_JRA2.assign_coords(year = trans_hov_LC_south_JRA2.year.values + 44)
print(trans_hov_LC_south_JRA2.year.values)



trans_hov_LC_south_JRA = xr.concat([trans_hov_LC_south_JRA1, trans_hov_LC_south_JRA2], dim='year')
print(trans_hov_LC_south_JRA)

trans_hov_LC_south_1_xr = xr.DataArray(trans_hov_LC_south_1, name='trans_hov_LC_south_1',
                        coords=[trans_hov_LC_south_1.year], 
                        dims=['year'])
print(trans_hov_LC_south_1_xr)
trans_hov_LC_south_1_xr.to_netcdf(output_path_mod + 'trans_hov_LC_south_1.nc')


trans_hov_LC_south_025_xr = xr.DataArray(trans_hov_LC_south_025, name='trans_hov_LC_south_025',
                        coords=[trans_hov_LC_south_025.year], 
                        dims=['year'])
print(trans_hov_LC_south_025_xr)
trans_hov_LC_south_025_xr.to_netcdf(output_path_mod + 'trans_hov_LC_south_025.nc')


trans_hov_LC_south_01_xr = xr.DataArray(trans_hov_LC_south_01, name='trans_hov_LC_south_01',
                        coords=[trans_hov_LC_south_01.year], 
                        dims=['year'])
print(trans_hov_LC_south_01_xr)
trans_hov_LC_south_01_xr.to_netcdf(output_path_mod + 'trans_hov_LC_south_01.nc')


trans_hov_LC_south_JRA_xr = xr.DataArray(trans_hov_LC_south_JRA, name='trans_hov_LC_south_JRA',
                        coords=[trans_hov_LC_south_JRA.year], 
                        dims=['year'])
print(trans_hov_LC_south_JRA_xr)
trans_hov_LC_south_JRA_xr.to_netcdf(output_path_mod + 'trans_hov_LC_south_JRA.nc')

In [16]:
ekey='1deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_line_157E_1 = cc.get_nc_variable(expt, 'ocean_month.nc', 'tx_trans_int_z',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xu_ocean=line_157E['line_157E Strait'][0], method='nearest').\
                        sel(yt_ocean=slice(
                                    line_157E['line_157E Strait'][2],
                                    line_157E['line_157E Strait'][3])).\
                        sum('yt_ocean')

trans_line_157E_1.load()

trans_line_157E_1 = trans_line_157E_1.groupby('time.year').mean('time', skipna=True)
print(trans_line_157E_1)


ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_line_157E_025 = cc.get_nc_variable(expt, 'ocean_month.nc', 'tx_trans_int_z',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xu_ocean=line_157E['line_157E Strait'][0], method='nearest').\
                        sel(yt_ocean=slice(
                                    line_157E['line_157E Strait'][2],
                                    line_157E['line_157E Strait'][3])).\
                        sum('yt_ocean')

trans_line_157E_025.load()

trans_line_157E_025 = trans_line_157E_025.groupby('time.year').mean('time', skipna=True)
print(trans_line_157E_025)


ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_line_157E_01_month = cc.get_nc_variable(expt, 'ocean_month.nc', 'tx_trans_int_z',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xu_ocean=line_157E['line_157E Strait'][0], method='nearest').\
                        sel(yt_ocean=slice(
                                    line_157E['line_157E Strait'][2],
                                    line_157E['line_157E Strait'][3])).\
                        sum('yt_ocean')

trans_line_157E_01_month.load()
print(trans_line_157E_01_month)

trans_line_157E_01 = trans_line_157E_01_month.groupby('time.year').mean('time', skipna=True)
print(trans_line_157E_01)


time_units = 'days since 0001-01-01'
print(time_units)

trans_line_157E_JRA_month = cc.get_nc_variable('01deg_jra55v13_ryf8485_spinup6', 'ocean_month.nc', 'tx_trans_int_z',
                        time_units=time_units).\
                        sel(xu_ocean=line_157E['line_157E Strait'][0], method='nearest').\
                        sel(yt_ocean=slice(
                                    line_157E['line_157E Strait'][2],
                                    line_157E['line_157E Strait'][3])).\
                        sum('yt_ocean')

trans_line_157E_JRA_month.load()



trans_line_157E_JRA1 = trans_line_157E_JRA_month.isel(time=slice(0,418)).\
                        groupby('time.year').mean('time', skipna=True)-273.15
trans_line_157E_JRA1 = trans_line_157E_JRA1.assign_coords(year = trans_line_157E_JRA1.year.values + 1944)
print(trans_line_157E_JRA1.year.values)



trans_line_157E_JRA2 = trans_line_157E_JRA_month.isel(time=slice(446,-8))
trans_line_157E_JRA2 = trans_line_157E_JRA2.assign_coords(time = pd.to_datetime(trans_line_157E_JRA2.time.values))
trans_line_157E_JRA2 = trans_line_157E_JRA2.groupby('time.year').mean('time', skipna=True)
print(trans_line_157E_JRA2.year.values)
trans_line_157E_JRA2 = trans_line_157E_JRA2.assign_coords(year = trans_line_157E_JRA2.year.values + 44)
print(trans_line_157E_JRA2.year.values)



trans_line_157E_JRA = xr.concat([trans_line_157E_JRA1, trans_line_157E_JRA2], dim='year')
print(trans_line_157E_JRA)

trans_line_157E_1_xr = xr.DataArray(trans_line_157E_1, name='trans_line_157E_1',
                        coords=[trans_line_157E_1.year], 
                        dims=['year'])
print(trans_line_157E_1_xr)
trans_line_157E_1_xr.to_netcdf(output_path_mod + 'trans_line_157E_1.nc')


trans_line_157E_025_xr = xr.DataArray(trans_line_157E_025, name='trans_line_157E_025',
                        coords=[trans_line_157E_025.year], 
                        dims=['year'])
print(trans_line_157E_025_xr)
trans_line_157E_025_xr.to_netcdf(output_path_mod + 'trans_line_157E_025.nc')


trans_line_157E_01_xr = xr.DataArray(trans_line_157E_01, name='trans_line_157E_01',
                        coords=[trans_line_157E_01.year], 
                        dims=['year'])
print(trans_line_157E_01_xr)
trans_line_157E_01_xr.to_netcdf(output_path_mod + 'trans_line_157E_01.nc')


trans_line_157E_JRA_xr = xr.DataArray(trans_line_157E_JRA, name='trans_line_157E_JRA',
                        coords=[trans_line_157E_JRA.year], 
                        dims=['year'])
print(trans_line_157E_JRA_xr)
trans_line_157E_JRA_xr.to_netcdf(output_path_mod + 'trans_line_157E_JRA.nc')

1993-01-01 00:00:00
2018-01-01 00:00:00
1deg_jra55v13_iaf_spinup1_B1
-12
days since 1718-01-01
-87658
Using database sqlite:////g/data3/hh5/tmp/cosima/cosima-cookbook/cosima-cookbook.db
<xarray.DataArray 'tx_trans_int_z' (year: 300)>
array([-1.047197e+10, -9.629212e+09, -1.013856e+10, ..., -3.435272e+09,
       -5.054287e+09, -3.777028e+09], dtype=float32)
Coordinates:
    xu_ocean  float64 -203.0
  * year      (year) int64 1718 1719 1720 1721 1722 ... 2013 2014 2015 2016 2017
1993-01-01 00:00:00
2018-01-01 00:00:00
025deg_jra55v13_iaf_gmredi6
-34
days since 1718-01-01
-87658
Using database sqlite:////g/data3/hh5/tmp/cosima/cosima-cookbook/cosima-cookbook.db
<xarray.DataArray 'tx_trans_int_z' (year: 300)>
array([-1.135615e+10, -9.013612e+09, -4.687366e+09, ..., -7.467319e+09,
       -1.053404e+10, -1.023980e+10], dtype=float32)
Coordinates:
    xu_ocean  float64 -203.0
  * year      (year) int64 1718 1719 1720 1721 1722 ... 2013 2014 2015 2016 2017
1993-01-01 00:00:00
2018-01-01 00:00:

In [15]:
ekey='1deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_hov_EAC_north_1 = cc.get_nc_variable(expt, 'ocean_month.nc', 'ty_trans_int_z',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xt_ocean=slice(
                                    EAC['EAC_north Strait'][0],
                                    EAC['EAC_north Strait'][1])).\
                        sel(yu_ocean=EAC['EAC_north Strait'][2], method='nearest')

trans_hov_EAC_north_1.load()

trans_hov_EAC_north_1 = trans_hov_EAC_north_1.groupby('time.year').mean('time', skipna=True)
print(trans_hov_EAC_north_1)


ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_hov_EAC_north_025 = cc.get_nc_variable(expt, 'ocean_month.nc', 'ty_trans_int_z',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xt_ocean=slice(
                                    EAC['EAC_north Strait'][0],
                                    EAC['EAC_north Strait'][1])).\
                        sel(yu_ocean=EAC['EAC_north Strait'][2], method='nearest')

trans_hov_EAC_north_025.load()

trans_hov_EAC_north_025 = trans_hov_EAC_north_025.groupby('time.year').mean('time', skipna=True)
print(trans_hov_EAC_north_025)


ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_hov_EAC_north_01_month = cc.get_nc_variable(expt, 'ocean_month.nc', 'ty_trans_int_z',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xt_ocean=slice(
                                    EAC['EAC_north Strait'][0],
                                    EAC['EAC_north Strait'][1])).\
                        sel(yu_ocean=EAC['EAC_north Strait'][2], method='nearest')

trans_hov_EAC_north_01_month.load()
print(trans_hov_EAC_north_01_month)

trans_hov_EAC_north_01 = trans_hov_EAC_north_01_month.groupby('time.year').mean('time', skipna=True)
print(trans_hov_EAC_north_01)


time_units = 'days since 0001-01-01'
print(time_units)

trans_hov_EAC_north_JRA_month = cc.get_nc_variable('01deg_jra55v13_ryf8485_spinup6', 'ocean_month.nc', 'ty_trans_int_z',
                        time_units=time_units).\
                        sel(xt_ocean=slice(
                                    EAC['EAC_north Strait'][0],
                                    EAC['EAC_north Strait'][1])).\
                        sel(yu_ocean=EAC['EAC_north Strait'][2], method='nearest')

trans_hov_EAC_north_JRA_month.load()



trans_hov_EAC_north_JRA1 = trans_hov_EAC_north_JRA_month.isel(time=slice(0,418)).\
                        groupby('time.year').mean('time', skipna=True)
trans_hov_EAC_north_JRA1 = trans_hov_EAC_north_JRA1.assign_coords(year = trans_hov_EAC_north_JRA1.year.values + 1944)
print(trans_hov_EAC_north_JRA1.year.values)



trans_hov_EAC_north_JRA2 = trans_hov_EAC_north_JRA_month.isel(time=slice(446,-8))
trans_hov_EAC_north_JRA2 = trans_hov_EAC_north_JRA2.assign_coords(time = pd.to_datetime(trans_hov_EAC_north_JRA2.time.values))
trans_hov_EAC_north_JRA2 = trans_hov_EAC_north_JRA2.groupby('time.year').mean('time', skipna=True)
print(trans_hov_EAC_north_JRA2.year.values)
trans_hov_EAC_north_JRA2 = trans_hov_EAC_north_JRA2.assign_coords(year = trans_hov_EAC_north_JRA2.year.values + 44)
print(trans_hov_EAC_north_JRA2.year.values)



trans_hov_EAC_north_JRA = xr.concat([trans_hov_EAC_north_JRA1, trans_hov_EAC_north_JRA2], dim='year')
print(trans_hov_EAC_north_JRA)

trans_hov_EAC_north_1_xr = xr.DataArray(trans_hov_EAC_north_1, name='trans_hov_EAC_north_1',
                        coords=[trans_hov_EAC_north_1.year, trans_hov_EAC_north_1.xt_ocean], 
                        dims=['year', 'xt_ocean'])
print(trans_hov_EAC_north_1_xr)
trans_hov_EAC_north_1_xr.to_netcdf(output_path_mod + 'trans_hov_EAC_north_1.nc')


trans_hov_EAC_north_025_xr = xr.DataArray(trans_hov_EAC_north_025, name='trans_hov_EAC_north_025',
                        coords=[trans_hov_EAC_north_025.year, trans_hov_EAC_north_025.xt_ocean], 
                        dims=['year', 'xt_ocean'])
print(trans_hov_EAC_north_025_xr)
trans_hov_EAC_north_025_xr.to_netcdf(output_path_mod + 'trans_hov_EAC_north_025.nc')


trans_hov_EAC_north_01_xr = xr.DataArray(trans_hov_EAC_north_01, name='trans_hov_EAC_north_01',
                        coords=[trans_hov_EAC_north_01.year, trans_hov_EAC_north_01.xt_ocean], 
                        dims=['year', 'xt_ocean'])
print(trans_hov_EAC_north_01_xr)
trans_hov_EAC_north_01_xr.to_netcdf(output_path_mod + 'trans_hov_EAC_north_01.nc')


trans_hov_EAC_north_JRA_xr = xr.DataArray(trans_hov_EAC_north_JRA, name='trans_hov_EAC_north_JRA',
                        coords=[trans_hov_EAC_north_JRA.year, trans_hov_EAC_north_JRA.xt_ocean], 
                        dims=['year', 'xt_ocean'])
print(trans_hov_EAC_north_JRA_xr)
trans_hov_EAC_north_JRA_xr.to_netcdf(output_path_mod + 'trans_hov_EAC_north_JRA.nc')

1993-01-01 00:00:00
2018-01-01 00:00:00
1deg_jra55v13_iaf_spinup1_B1
-12
days since 1718-01-01
-87658
Using database sqlite:////g/data3/hh5/tmp/cosima/cosima-cookbook/cosima-cookbook.db


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-18.10/lib/python3.6/site-packages/xarray/core/nanops.py:161: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


<xarray.DataArray 'ty_trans_int_z' (year: 300, xt_ocean: 10)>
array([[          nan,           nan,           nan, ..., -6.559635e+09,
        -4.593583e+09, -1.263368e+09],
       [          nan,           nan,           nan, ..., -6.032843e+09,
        -4.420680e+09, -1.592285e+09],
       [          nan,           nan,           nan, ..., -5.783958e+09,
        -4.421826e+09, -1.774663e+09],
       ...,
       [          nan,           nan,           nan, ..., -5.066660e+09,
        -3.757353e+09, -1.480016e+09],
       [          nan,           nan,           nan, ..., -5.225220e+09,
        -3.842134e+09, -1.380969e+09],
       [          nan,           nan,           nan, ..., -4.236934e+09,
        -3.014642e+09, -9.282931e+08]], dtype=float32)
Coordinates:
    yu_ocean  float64 -33.0
  * xt_ocean  (xt_ocean) float64 -212.5 -211.5 -210.5 ... -205.5 -204.5 -203.5
  * year      (year) int64 1718 1719 1720 1721 1722 ... 2013 2014 2015 2016 2017
1993-01-01 00:00:00
2018-01-01 00:00:

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-18.10/lib/python3.6/site-packages/xarray/core/nanops.py:161: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


<xarray.DataArray 'ty_trans_int_z' (year: 300, xt_ocean: 40)>
array([[          nan,           nan,           nan, ...,  5.236506e+07,
         7.713442e+08,  1.805683e+09],
       [          nan,           nan,           nan, ..., -4.172615e+08,
        -4.154040e+08, -2.724100e+08],
       [          nan,           nan,           nan, ..., -2.513246e+08,
        -1.131347e+09, -2.061421e+09],
       ...,
       [          nan,           nan,           nan, ..., -8.606214e+08,
        -1.591060e+09, -1.490425e+09],
       [          nan,           nan,           nan, ..., -2.870087e+08,
        -4.042793e+08, -1.887471e+08],
       [          nan,           nan,           nan, ..., -6.173273e+08,
        -9.992775e+08, -7.169297e+08]], dtype=float32)
Coordinates:
  * xt_ocean  (xt_ocean) float64 -212.9 -212.6 -212.4 ... -203.6 -203.4 -203.1
    yu_ocean  float64 -33.01
  * year      (year) int64 1718 1719 1720 1721 1722 ... 2013 2014 2015 2016 2017
1993-01-01 00:00:00
2018-01-01 00:00

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-18.10/lib/python3.6/site-packages/xarray/core/nanops.py:161: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


[1945 1946 1947 1948 1949 1950 1951 1952 1953 1954 1955 1956 1957 1958
 1959 1960 1961 1962 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972
 1973 1974 1975 1976 1977 1978 1979]
[1935 1936 1937 1938 1939 1940 1941 1942 1943 1944 1945]
[1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989]
<xarray.DataArray 'ty_trans_int_z' (year: 46, xt_ocean: 100)>
array([[          nan,           nan,           nan, ..., -2.855263e+07,
        -2.453831e+07,  1.216687e+08],
       [          nan,           nan,           nan, ..., -2.490448e+08,
        -2.429472e+08, -1.335263e+08],
       [          nan,           nan,           nan, ...,  1.333882e+08,
         1.089166e+08,  1.548832e+08],
       ...,
       [          nan,           nan,           nan, ..., -6.876207e+08,
        -5.765377e+08, -2.899390e+08],
       [          nan,           nan,           nan, ..., -2.949690e+08,
        -3.469421e+08, -2.439235e+08],
       [          nan,           nan,           nan, ..., -4.464166e+07,


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-18.10/lib/python3.6/site-packages/xarray/core/nanops.py:161: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


ValueError: conflicting sizes for dimension 'year': length 300 on the data but length 10 on coordinate 'year'

In [16]:
trans_hov_EAC_north_JRA = xr.concat([trans_hov_EAC_north_JRA1, trans_hov_EAC_north_JRA2], dim='year')
print(trans_hov_EAC_north_JRA)

trans_hov_EAC_north_1_xr = xr.DataArray(trans_hov_EAC_north_1, name='trans_hov_EAC_north_1',
                        coords=[trans_hov_EAC_north_1.year, trans_hov_EAC_north_1.xt_ocean], 
                        dims=['year', 'xt_ocean'])
print(trans_hov_EAC_north_1_xr)
trans_hov_EAC_north_1_xr.to_netcdf(output_path_mod + 'trans_hov_EAC_north_1.nc')


trans_hov_EAC_north_025_xr = xr.DataArray(trans_hov_EAC_north_025, name='trans_hov_EAC_north_025',
                        coords=[trans_hov_EAC_north_025.year, trans_hov_EAC_north_025.xt_ocean], 
                        dims=['year', 'xt_ocean'])
print(trans_hov_EAC_north_025_xr)
trans_hov_EAC_north_025_xr.to_netcdf(output_path_mod + 'trans_hov_EAC_north_025.nc')


trans_hov_EAC_north_01_xr = xr.DataArray(trans_hov_EAC_north_01, name='trans_hov_EAC_north_01',
                        coords=[trans_hov_EAC_north_01.year, trans_hov_EAC_north_01.xt_ocean], 
                        dims=['year', 'xt_ocean'])
print(trans_hov_EAC_north_01_xr)
trans_hov_EAC_north_01_xr.to_netcdf(output_path_mod + 'trans_hov_EAC_north_01.nc')


trans_hov_EAC_north_JRA_xr = xr.DataArray(trans_hov_EAC_north_JRA, name='trans_hov_EAC_north_JRA',
                        coords=[trans_hov_EAC_north_JRA.year, trans_hov_EAC_north_JRA.xt_ocean], 
                        dims=['year', 'xt_ocean'])
print(trans_hov_EAC_north_JRA_xr)
trans_hov_EAC_north_JRA_xr.to_netcdf(output_path_mod + 'trans_hov_EAC_north_JRA.nc')

<xarray.DataArray 'ty_trans_int_z' (year: 46, xt_ocean: 100)>
array([[          nan,           nan,           nan, ..., -2.855263e+07,
        -2.453831e+07,  1.216687e+08],
       [          nan,           nan,           nan, ..., -2.490448e+08,
        -2.429472e+08, -1.335263e+08],
       [          nan,           nan,           nan, ...,  1.333882e+08,
         1.089166e+08,  1.548832e+08],
       ...,
       [          nan,           nan,           nan, ..., -6.876207e+08,
        -5.765377e+08, -2.899390e+08],
       [          nan,           nan,           nan, ..., -2.949690e+08,
        -3.469421e+08, -2.439235e+08],
       [          nan,           nan,           nan, ..., -4.464166e+07,
        -4.138829e+07,  6.362861e+07]], dtype=float32)
Coordinates:
    yu_ocean  float64 -33.01
  * xt_ocean  (xt_ocean) float64 -212.9 -212.8 -212.7 ... -203.2 -203.1 -203.0
  * year      (year) int64 1945 1946 1947 1948 1949 ... 1985 1986 1987 1988 1989
<xarray.DataArray 'trans_hov_EAC_nor

In [20]:
ekey='1deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_EAC_east_1 = cc.get_nc_variable(expt, 'ocean_month.nc', 'tx_trans_int_z',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xu_ocean=EAC['EAC_east Strait'][0], method='nearest').\
                        sel(yt_ocean=slice(
                                    EAC['EAC_east Strait'][2],
                                    EAC['EAC_east Strait'][3])).\
                        sum('yt_ocean')

trans_EAC_east_1.load()

trans_EAC_east_1 = trans_EAC_east_1.groupby('time.year').mean('time', skipna=True)
print(trans_EAC_east_1)


ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_EAC_east_025 = cc.get_nc_variable(expt, 'ocean_month.nc', 'tx_trans_int_z',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xu_ocean=EAC['EAC_east Strait'][0], method='nearest').\
                        sel(yt_ocean=slice(
                                    EAC['EAC_east Strait'][2],
                                    EAC['EAC_east Strait'][3])).\
                        sum('yt_ocean')

trans_EAC_east_025.load()

trans_EAC_east_025 = trans_EAC_east_025.groupby('time.year').mean('time', skipna=True)
print(trans_EAC_east_025)


ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_EAC_east_01_month = cc.get_nc_variable(expt, 'ocean_month.nc', 'tx_trans_int_z',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xu_ocean=EAC['EAC_east Strait'][0], method='nearest').\
                        sel(yt_ocean=slice(
                                    EAC['EAC_east Strait'][2],
                                    EAC['EAC_east Strait'][3])).\
                        sum('yt_ocean')

trans_EAC_east_01_month.load()
print(trans_EAC_east_01_month)

trans_EAC_east_01 = trans_EAC_east_01_month.groupby('time.year').mean('time', skipna=True)
print(trans_EAC_east_01)


time_units = 'days since 0001-01-01'
print(time_units)

trans_EAC_east_JRA_month = cc.get_nc_variable('01deg_jra55v13_ryf8485_spinup6', 'ocean_month.nc', 'tx_trans_int_z',
                        time_units=time_units).\
                        sel(xu_ocean=EAC['EAC_east Strait'][0], method='nearest').\
                        sel(yt_ocean=slice(
                                    EAC['EAC_east Strait'][2],
                                    EAC['EAC_east Strait'][3])).\
                        sum('yt_ocean')

trans_EAC_east_JRA_month.load()



trans_EAC_east_JRA1 = trans_EAC_east_JRA_month.isel(time=slice(0,418)).\
                        groupby('time.year').mean('time', skipna=True)-273.15
trans_EAC_east_JRA1 = trans_EAC_east_JRA1.assign_coords(year = trans_EAC_east_JRA1.year.values + 1944)
print(trans_EAC_east_JRA1.year.values)



trans_EAC_east_JRA2 = trans_EAC_east_JRA_month.isel(time=slice(446,-8))
trans_EAC_east_JRA2 = trans_EAC_east_JRA2.assign_coords(time = pd.to_datetime(trans_EAC_east_JRA2.time.values))
trans_EAC_east_JRA2 = trans_EAC_east_JRA2.groupby('time.year').mean('time', skipna=True)
print(trans_EAC_east_JRA2.year.values)
trans_EAC_east_JRA2 = trans_EAC_east_JRA2.assign_coords(year = trans_EAC_east_JRA2.year.values + 44)
print(trans_EAC_east_JRA2.year.values)



trans_EAC_east_JRA = xr.concat([trans_EAC_east_JRA1, trans_EAC_east_JRA2], dim='year')
print(trans_EAC_east_JRA)

trans_EAC_east_1_xr = xr.DataArray(trans_EAC_east_1, name='trans_EAC_east_1',
                        coords=[trans_EAC_east_1.year], 
                        dims=['year'])
print(trans_EAC_east_1_xr)
trans_EAC_east_1_xr.to_netcdf(output_path_mod + 'trans_EAC_east_1.nc')


trans_EAC_east_025_xr = xr.DataArray(trans_EAC_east_025, name='trans_EAC_east_025',
                        coords=[trans_EAC_east_025.year], 
                        dims=['year'])
print(trans_EAC_east_025_xr)
trans_EAC_east_025_xr.to_netcdf(output_path_mod + 'trans_EAC_east_025.nc')


trans_EAC_east_01_xr = xr.DataArray(trans_EAC_east_01, name='trans_EAC_east_01',
                        coords=[trans_EAC_east_01.year], 
                        dims=['year'])
print(trans_EAC_east_01_xr)
trans_EAC_east_01_xr.to_netcdf(output_path_mod + 'trans_EAC_east_01.nc')


trans_EAC_east_JRA_xr = xr.DataArray(trans_EAC_east_JRA, name='trans_EAC_east_JRA',
                        coords=[trans_EAC_east_JRA.year], 
                        dims=['year'])
print(trans_EAC_east_JRA_xr)
trans_EAC_east_JRA_xr.to_netcdf(output_path_mod + 'trans_EAC_east_JRA.nc')

1993-01-01 00:00:00
2018-01-01 00:00:00
1deg_jra55v13_iaf_spinup1_B1
-12
days since 1718-01-01
-87658
Using database sqlite:////g/data3/hh5/tmp/cosima/cosima-cookbook/cosima-cookbook.db
<xarray.DataArray 'tx_trans_int_z' (year: 300)>
array([1.098755e+10, 8.205873e+09, 7.145619e+09, ..., 5.320758e+09,
       5.344422e+09, 4.906110e+09], dtype=float32)
Coordinates:
    xu_ocean  float64 -203.0
  * year      (year) int64 1718 1719 1720 1721 1722 ... 2013 2014 2015 2016 2017
1993-01-01 00:00:00
2018-01-01 00:00:00
025deg_jra55v13_iaf_gmredi6
-34
days since 1718-01-01
-87658
Using database sqlite:////g/data3/hh5/tmp/cosima/cosima-cookbook/cosima-cookbook.db
<xarray.DataArray 'tx_trans_int_z' (year: 300)>
array([1.199599e+10, 5.314361e+09, 5.791508e+08, ..., 2.590437e+09,
       4.503590e+09, 6.604740e+09], dtype=float32)
Coordinates:
    xu_ocean  float64 -203.0
  * year      (year) int64 1718 1719 1720 1721 1722 ... 2013 2014 2015 2016 2017
1993-01-01 00:00:00
2018-01-01 00:00:00
01deg_jra

In [21]:
ekey='1deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_EAC_south_1 = cc.get_nc_variable(expt, 'ocean_month.nc', 'ty_trans_int_z',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xt_ocean=slice(
                                    EAC['EAC_south Strait'][0],
                                    EAC['EAC_south Strait'][1])).\
                        sel(yu_ocean=EAC['EAC_south Strait'][2], method='nearest').\
                        sum('xt_ocean')

trans_EAC_south_1.load()

trans_EAC_south_1 = trans_EAC_south_1.groupby('time.year').mean('time', skipna=True)
print(trans_EAC_south_1)


ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_EAC_south_025 = cc.get_nc_variable(expt, 'ocean_month.nc', 'ty_trans_int_z',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xt_ocean=slice(
                                    EAC['EAC_south Strait'][0],
                                    EAC['EAC_south Strait'][1])).\
                        sel(yu_ocean=EAC['EAC_south Strait'][2], method='nearest').\
                        sum('xt_ocean')

trans_EAC_south_025.load()

trans_EAC_south_025 = trans_EAC_south_025.groupby('time.year').mean('time', skipna=True)
print(trans_EAC_south_025)


ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_EAC_south_01_month = cc.get_nc_variable(expt, 'ocean_month.nc', 'ty_trans_int_z',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xt_ocean=slice(
                                    EAC['EAC_south Strait'][0],
                                    EAC['EAC_south Strait'][1])).\
                        sel(yu_ocean=EAC['EAC_south Strait'][2], method='nearest').\
                        sum('xt_ocean')

trans_EAC_south_01_month.load()
print(trans_EAC_south_01_month)

trans_EAC_south_01 = trans_EAC_south_01_month.groupby('time.year').mean('time', skipna=True)
print(trans_EAC_south_01)


time_units = 'days since 0001-01-01'
print(time_units)

trans_EAC_south_JRA_month = cc.get_nc_variable('01deg_jra55v13_ryf8485_spinup6', 'ocean_month.nc', 'ty_trans_int_z',
                        time_units=time_units).\
                        sel(xt_ocean=slice(
                                    EAC['EAC_south Strait'][0],
                                    EAC['EAC_south Strait'][1])).\
                        sel(yu_ocean=EAC['EAC_south Strait'][2], method='nearest').\
                        sum('xt_ocean')

trans_EAC_south_JRA_month.load()



trans_EAC_south_JRA1 = trans_EAC_south_JRA_month.isel(time=slice(0,418)).\
                        groupby('time.year').mean('time', skipna=True)
trans_EAC_south_JRA1 = trans_EAC_south_JRA1.assign_coords(year = trans_EAC_south_JRA1.year.values + 1944)
print(trans_EAC_south_JRA1.year.values)



trans_EAC_south_JRA2 = trans_EAC_south_JRA_month.isel(time=slice(446,-8))
trans_EAC_south_JRA2 = trans_EAC_south_JRA2.assign_coords(time = pd.to_datetime(trans_EAC_south_JRA2.time.values))
trans_EAC_south_JRA2 = trans_EAC_south_JRA2.groupby('time.year').mean('time', skipna=True)
print(trans_EAC_south_JRA2.year.values)
trans_EAC_south_JRA2 = trans_EAC_south_JRA2.assign_coords(year = trans_EAC_south_JRA2.year.values + 44)
print(trans_EAC_south_JRA2.year.values)



trans_EAC_south_JRA = xr.concat([trans_EAC_south_JRA1, trans_EAC_south_JRA2], dim='year')
print(trans_EAC_south_JRA)

trans_EAC_south_1_xr = xr.DataArray(trans_EAC_south_1, name='trans_EAC_south_1',
                        coords=[trans_EAC_south_1.year], 
                        dims=['year'])
print(trans_EAC_south_1_xr)
trans_EAC_south_1_xr.to_netcdf(output_path_mod + 'trans_EAC_south_1.nc')


trans_EAC_south_025_xr = xr.DataArray(trans_EAC_south_025, name='trans_EAC_south_025',
                        coords=[trans_EAC_south_025.year], 
                        dims=['year'])
print(trans_EAC_south_025_xr)
trans_EAC_south_025_xr.to_netcdf(output_path_mod + 'trans_EAC_south_025.nc')


trans_EAC_south_01_xr = xr.DataArray(trans_EAC_south_01, name='trans_EAC_south_01',
                        coords=[trans_EAC_south_01.year], 
                        dims=['year'])
print(trans_EAC_south_01_xr)
trans_EAC_south_01_xr.to_netcdf(output_path_mod + 'trans_EAC_south_01.nc')


trans_EAC_south_JRA_xr = xr.DataArray(trans_EAC_south_JRA, name='trans_EAC_south_JRA',
                        coords=[trans_EAC_south_JRA.year], 
                        dims=['year'])
print(trans_EAC_south_JRA_xr)
trans_EAC_south_JRA_xr.to_netcdf(output_path_mod + 'trans_EAC_south_JRA.nc')

1993-01-01 00:00:00
2018-01-01 00:00:00
1deg_jra55v13_iaf_spinup1_B1
-12
days since 1718-01-01
-87658
Using database sqlite:////g/data3/hh5/tmp/cosima/cosima-cookbook/cosima-cookbook.db
<xarray.DataArray 'ty_trans_int_z' (year: 300)>
array([ 2.583187e+09,  3.319557e+09,  2.105608e+09, ..., -4.273216e+06,
       -1.548754e+09,  4.653577e+08], dtype=float32)
Coordinates:
    yu_ocean  float64 -47.41
  * year      (year) int64 1718 1719 1720 1721 1722 ... 2013 2014 2015 2016 2017
1993-01-01 00:00:00
2018-01-01 00:00:00
025deg_jra55v13_iaf_gmredi6
-34
days since 1718-01-01
-87658
Using database sqlite:////g/data3/hh5/tmp/cosima/cosima-cookbook/cosima-cookbook.db
<xarray.DataArray 'ty_trans_int_z' (year: 300)>
array([ 3.184639e+09,  4.236280e+09,  8.308145e+08, ..., -5.657065e+08,
        2.064130e+09,  4.473356e+09], dtype=float32)
Coordinates:
    yu_ocean  float64 -47.08
  * year      (year) int64 1718 1719 1720 1721 1722 ... 2013 2014 2015 2016 2017
1993-01-01 00:00:00
2018-01-01 00:00:

In [22]:
ekey='1deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_EAC_west_1 = cc.get_nc_variable(expt, 'ocean_month.nc', 'tx_trans_int_z',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xu_ocean=EAC['EAC_west Strait'][0], method='nearest').\
                        sel(yt_ocean=slice(
                                    EAC['EAC_west Strait'][2],
                                    EAC['EAC_west Strait'][3])).\
                        sum('yt_ocean')

trans_EAC_west_1.load()

trans_EAC_west_1 = trans_EAC_west_1.groupby('time.year').mean('time', skipna=True)
print(trans_EAC_west_1)


ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_EAC_west_025 = cc.get_nc_variable(expt, 'ocean_month.nc', 'tx_trans_int_z',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xu_ocean=EAC['EAC_west Strait'][0], method='nearest').\
                        sel(yt_ocean=slice(
                                    EAC['EAC_west Strait'][2],
                                    EAC['EAC_west Strait'][3])).\
                        sum('yt_ocean')

trans_EAC_west_025.load()

trans_EAC_west_025 = trans_EAC_west_025.groupby('time.year').mean('time', skipna=True)
print(trans_EAC_west_025)


ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
print(tstart)
print(tend)
print(expt)
print(n_files)
print(time_units)
print(offset)

trans_EAC_west_01_month = cc.get_nc_variable(expt, 'ocean_month.nc', 'tx_trans_int_z',
                        n=None,
                        time_units=time_units, offset=offset).\
                        sel(xu_ocean=EAC['EAC_west Strait'][0], method='nearest').\
                        sel(yt_ocean=slice(
                                    EAC['EAC_west Strait'][2],
                                    EAC['EAC_west Strait'][3])).\
                        sum('yt_ocean')

trans_EAC_west_01_month.load()
print(trans_EAC_west_01_month)

trans_EAC_west_01 = trans_EAC_west_01_month.groupby('time.year').mean('time', skipna=True)
print(trans_EAC_west_01)


time_units = 'days since 0001-01-01'
print(time_units)

trans_EAC_west_JRA_month = cc.get_nc_variable('01deg_jra55v13_ryf8485_spinup6', 'ocean_month.nc', 'tx_trans_int_z',
                        time_units=time_units).\
                        sel(xu_ocean=EAC['EAC_west Strait'][0], method='nearest').\
                        sel(yt_ocean=slice(
                                    EAC['EAC_west Strait'][2],
                                    EAC['EAC_west Strait'][3])).\
                        sum('yt_ocean')

trans_EAC_west_JRA_month.load()



trans_EAC_west_JRA1 = trans_EAC_west_JRA_month.isel(time=slice(0,418)).\
                        groupby('time.year').mean('time', skipna=True)-273.15
trans_EAC_west_JRA1 = trans_EAC_west_JRA1.assign_coords(year = trans_EAC_west_JRA1.year.values + 1944)
print(trans_EAC_west_JRA1.year.values)



trans_EAC_west_JRA2 = trans_EAC_west_JRA_month.isel(time=slice(446,-8))
trans_EAC_west_JRA2 = trans_EAC_west_JRA2.assign_coords(time = pd.to_datetime(trans_EAC_west_JRA2.time.values))
trans_EAC_west_JRA2 = trans_EAC_west_JRA2.groupby('time.year').mean('time', skipna=True)
print(trans_EAC_west_JRA2.year.values)
trans_EAC_west_JRA2 = trans_EAC_west_JRA2.assign_coords(year = trans_EAC_west_JRA2.year.values + 44)
print(trans_EAC_west_JRA2.year.values)



trans_EAC_west_JRA = xr.concat([trans_EAC_west_JRA1, trans_EAC_west_JRA2], dim='year')
print(trans_EAC_west_JRA)

trans_EAC_west_1_xr = xr.DataArray(trans_EAC_west_1, name='trans_EAC_west_1',
                        coords=[trans_EAC_west_1.year], 
                        dims=['year'])
print(trans_EAC_west_1_xr)
trans_EAC_west_1_xr.to_netcdf(output_path_mod + 'trans_EAC_west_1.nc')


trans_EAC_west_025_xr = xr.DataArray(trans_EAC_west_025, name='trans_EAC_west_025',
                        coords=[trans_EAC_west_025.year], 
                        dims=['year'])
print(trans_EAC_west_025_xr)
trans_EAC_west_025_xr.to_netcdf(output_path_mod + 'trans_EAC_west_025.nc')


trans_EAC_west_01_xr = xr.DataArray(trans_EAC_west_01, name='trans_EAC_west_01',
                        coords=[trans_EAC_west_01.year], 
                        dims=['year'])
print(trans_EAC_west_01_xr)
trans_EAC_west_01_xr.to_netcdf(output_path_mod + 'trans_EAC_west_01.nc')


trans_EAC_west_JRA_xr = xr.DataArray(trans_EAC_west_JRA, name='trans_EAC_west_JRA',
                        coords=[trans_EAC_west_JRA.year], 
                        dims=['year'])
print(trans_EAC_west_JRA_xr)
trans_EAC_west_JRA_xr.to_netcdf(output_path_mod + 'trans_EAC_west_JRA.nc')

1993-01-01 00:00:00
2018-01-01 00:00:00
1deg_jra55v13_iaf_spinup1_B1
-12
days since 1718-01-01
-87658
Using database sqlite:////g/data3/hh5/tmp/cosima/cosima-cookbook/cosima-cookbook.db
<xarray.DataArray 'tx_trans_int_z' (year: 300)>
array([-8.241926e+09, -1.087662e+10, -1.044660e+10, ..., -8.137404e+09,
       -6.988005e+09, -6.548766e+09], dtype=float32)
Coordinates:
    xu_ocean  float64 -213.0
  * year      (year) int64 1718 1719 1720 1721 1722 ... 2013 2014 2015 2016 2017
1993-01-01 00:00:00
2018-01-01 00:00:00
025deg_jra55v13_iaf_gmredi6
-34
days since 1718-01-01
-87658
Using database sqlite:////g/data3/hh5/tmp/cosima/cosima-cookbook/cosima-cookbook.db
<xarray.DataArray 'tx_trans_int_z' (year: 300)>
array([-8.845709e+09, -1.260371e+10, -8.415226e+09, ..., -1.189604e+10,
       -1.303109e+10, -1.145009e+10], dtype=float32)
Coordinates:
    xu_ocean  float64 -213.0
  * year      (year) int64 1718 1719 1720 1721 1722 ... 2013 2014 2015 2016 2017
1993-01-01 00:00:00
2018-01-01 00:00: